In [1]:
# import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser

In [38]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# saving urls of all sites to be scraped
news_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
images_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
weather_url = 'https://twitter.com/marswxreport?lang=en'
facts_url = 'https://space-facts.com/mars/'
astro_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

### NASA Mars News

In [ ]:
# setting 
# browser.visit(news_url)
# news_soup = bs(browser.html, 'html.parser')
# news_title = news_soup.find_all('div', class_='content_title')[0].text
# news_p = news_soup.find_all('div', class_='article_teaser_body')[0].text
# print(news_title)
# print(news_p)

### JPL Mars Space Images - Featured Image

In [ ]:
# find featured_image_url
# image_base_url = 'https://www.jpl.nasa.gov'
# browser.visit(images_url)
# browser.click_link_by_partial_text('FULL IMAGE')
# image_soup = bs(browser.html, 'html.parser')
# img = image_soup.find('img', class_='fancybox-image')['src']

# # combining base url with incomplete url from scraped featured img src
# featured_image_url = image_base_url + img
# featured_image_url

### Mars Weather

In [ ]:
# scrape latest weather tweet
# browser.visit(weather_url)
# weather_soup = bs(browser.html, 'html.parser')
# latest_weather_tweet = weather_soup.find('p', class_='TweetTextSize').text
# latest_weather_tweet

### Mars Facts

In [ ]:
# reading html into a dataframe
# facts_table = pd.read_html(facts_url)
# mars_table = facts_table[1]

# # renaming columns
# facts_mapping = {0:'Specifications', 1:'Measurements'}
# mars_table = mars_table.rename(columns=facts_mapping)

# # saving as html table format
# mars_table.to_html('mars_table.html', index=False)

### Mars Hemispheres

In [40]:
# scrape hemisphere images
browser.visit(astro_url)
astro_soup = bs(browser.html, 'html.parser')
astro_base_url = "https://astrogeology.usgs.gov"

image_list = []
titles = []
div_list = astro_soup.find_all('a', class_='itemLink')

for link in div_list:
    image_url = link.get('href')
    if image_url not in image_list:
        image_list.append(image_url)
        browser.visit(astro_base_url + image_url)
        titles.append(browser.find_by_tag('h2').text)
        browser.find_link_by_text('Sample').click()
        print(browser.url)

https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=76.0.3809.100)
  (Driver info: chromedriver=2.34.522940 (1a76f96f66e3ca7b8e57d503b4dd3bccfba87af1),platform=Windows NT 10.0.18362 x86_64)


['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']